In [545]:
# IMPORTS
import random as rnd
import numpy as np
import statistics as st

In [546]:
def debug_print(lista, titulo):
    print(titulo)
    for i in lista:
        print(i.fitness)

In [547]:
class individuo:
    def __init__(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.fitness = 0
    
    def set_fitness(self, f):
        self.fitness = f
        #print(self.fitness)
        
    def debug(self):
        s = "x1: "+ str(self.x1) + "\n" + "x2: "+ str(self.x2) + "\n" + "fitness: "+ str(self.fitness)
        return s
        

In [548]:
def pool_inicial(n_individuos, limsx1, limix1, limsx2, limix2, restr):
    population = []
    for i in range(n_individuos):
        x1 = round(rnd.random()*(limsx1 - limix1)+limix1, 1)
        limrx2 = restr(x1)
        limsx2 = min(limsx2,limrx2)
        x2 = round(rnd.random()*(limsx2 - limix2)+limix2, 1)
        ind = individuo(x1,x2)
        population.append(ind)
    #print("pool inicial", population)
    return population

In [549]:
def check_mutation(x1, x2, limsx1, limix1, limsx2, limix2, restr):
    check_x1 = False
    check_x2 = False
    limrx2 = restr(x1)
    limsx2 = min(limsx2,limrx2)
    if(x1<limsx1 and x1>limix1):
        check_x1 = True
    if(check_x1):
        limrx2 = restr(x1)
        limsx2 = min(limsx2,limrx2)
        if(x2<limsx2 and x2>limix2):
            check_x2 = True
        
    return check_x1, check_x2

In [550]:
def fitness_calculation(population, fun):
    for i in population:
        #print("indv x1: ", i.x1, "indv x2: ", i.x2)
        i.set_fitness(fun(i.x1, i.x2))

In [551]:
def selection(population):
    pop_sorted = sorted(population, key=lambda x: x.fitness, reverse=True)
    #debug_print(pop_sorted, "sorted population: ")
    new_population = pop_sorted[:round(len(pop_sorted)/2)]
    #debug_print(new_population,"selection return: ")
    return new_population

In [552]:
def cross(population):
    new_population = []
    for i in population:
        for j in population:
            if(i != j):
                x1_heredada = i.x1
                x2_heredada = j.x2
                hijo = individuo(x1_heredada, x2_heredada)
                new_population.append(hijo)
    #new_population.append(individuo(population[-1].x1, population[0].x2))
    return new_population[:len(population)*2]

In [553]:
def mutation(population, limsx1, limix1, limsx2, limix2, restr):
    new_population = []
    for i in population:
        c_x1 = False
        c_x2 = False
        while(not(c_x1) and not(c_x2)):
            gen_mutar = rnd.randint(0,1)
            if(gen_mutar == 0):
                x1 = round(i.x1 + rnd.gauss(0, 0.1), 3)
                x2 = i.x2
            else:
                x1 = i.x1
                x2 = round(i.x2 + rnd.gauss(0, 0.1), 3)
            c = check_mutation(x1, x2, limsx1, limix1, limsx2, limix2, restr)
            c_x1 = c[0]
            c_x2 = c[1]
        new_population.append(individuo(x1,x2))
    return new_population

In [554]:
def Optimo(fitness, n_individuos, limsx1, limix1, limsx2, limix2, restr, nmax):
    varianza = 1
    n = 0
    population = pool_inicial(n_individuos, limsx1, limix1, limsx2, limix2, restr)
    fitness_calculation(population, fitness)
    while(n < nmax):
        print("\n----> iteración: ", n)
        print("cant ind", len(population))
        print("first item: ", population[len(population)-1].x1, " ", population[len(population)-1].x2)
        population = selection(population)
        #debug_print(population, "after selection")
        population = cross(population)
        #debug_print(population, "after cross")
        population = mutation(population, limsx1, limix1, limsx2, limix2, restr)
        fitness_calculation(population, fitness)
        #debug_print(population, "after mutation")
        n += 1
        print("fitness order first item: ", population[0].fitness)
    return population[0]

In [555]:
fitness = lambda x1,x2: -(x1)**2 - (x2)**2 + 5
restr = lambda x1: x1
pos = np.linspace(-5,5,100)

o = Optimo(fitness, 30, 5, -5, 5, -5, restr, 100)
print("RESULTADOS: ")
print("x1: ",o.x1, "\tx2: ",o.x2)
print("fitness: ", o.fitness)


----> iteración:  0
cant ind 30
first item:  0.8   -4.7
fitness order first item:  -1.326224999999999

----> iteración:  1
cant ind 30
first item:  2.1   -2.516
fitness order first item:  3.9293560000000003

----> iteración:  2
cant ind 30
first item:  0.571   -0.851
fitness order first item:  4.146918

----> iteración:  3
cant ind 30
first item:  -0.7   -0.749
fitness order first item:  4.187623

----> iteración:  4
cant ind 30
first item:  0.537   -0.893
fitness order first item:  4.442094

----> iteración:  5
cant ind 30
first item:  0.557   -0.686
fitness order first item:  4.759255

----> iteración:  6
cant ind 30
first item:  0.213   -0.252
fitness order first item:  4.903007

----> iteración:  7
cant ind 30
first item:  0.377   -0.252
fitness order first item:  4.82655

----> iteración:  8
cant ind 30
first item:  0.353   -0.353
fitness order first item:  4.944022

----> iteración:  9
cant ind 30
first item:  0.374   -0.252
fitness order first item:  4.913883

----> iteración: 

# Task 1
## Función: 
$$f(x_1,x_2) = 15x_1 + 30x_2 + 4x_1x_2 - 2x_1^2 - 4x_2^3$$

## Restricciones: 
$$x_1 + 2x_2 \leq 30$$
$$x_1 \geq 0, x_2 \geq 0$$

In [565]:
fitness = lambda x1,x2: (15*x1)+(30*x2)+(4*x1*x2)-(2*(x1**2))-(4*(x2**2))
restr = lambda x1: 15 - (x1/2)
o = Optimo(fitness, 15, 40, 0, 40, 0, restr, 100)
print("RESULTADOS: ")
print("x1: ",o.x1, "\tx2: ",o.x2)
print("fitness: ", o.fitness)


----> iteración:  0
cant ind 15
first item:  22.4   -2.4
fitness order first item:  -348.64774399999993

----> iteración:  1
cant ind 16
first item:  10.502   -1.1
fitness order first item:  218.2338

----> iteración:  2
cant ind 16
first item:  1.978   5.792
fitness order first item:  222.77732400000002

----> iteración:  3
cant ind 16
first item:  1.978   5.85
fitness order first item:  220.25254399999994

----> iteración:  4
cant ind 16
first item:  10.379   5.792
fitness order first item:  221.32411000000002

----> iteración:  5
cant ind 16
first item:  10.329   6.089
fitness order first item:  227.71462000000002

----> iteración:  6
cant ind 16
first item:  10.205   6.089
fitness order first item:  227.044278

----> iteración:  7
cant ind 16
first item:  10.205   6.13
fitness order first item:  231.11315000000005

----> iteración:  8
cant ind 16
first item:  10.239   6.269
fitness order first item:  233.63812600000003

----> iteración:  9
cant ind 16
first item:  10.092   6.371
f

# Task 2
## Función:
$$z = 3x_1 + 5x_2$$
## Restricciones:
$$x_1 \leq 4$$
$$2x_2 \leq 12$$
$$3x_1 + 2x_2 \leq 18$$
$$x_1 >= 0, x_2 \geq 0$$

In [562]:
fitness = lambda x1,x2: (3*x1) + (5*x2)
restr = lambda x1: (18.0/2.0) - (3.0/2.0)*x1
o = Optimo(fitness, 15, 4, 0, 6, 0, restr, 100)
print("RESULTADOS: ")
print("x1: ",o.x1, "\tx2: ",o.x2)
print("fitness: ", o.fitness)


----> iteración:  0
cant ind 15
first item:  3.6   0.2
fitness order first item:  23.305

----> iteración:  1
cant ind 16
first item:  2.5   3.643
fitness order first item:  33.69

----> iteración:  2
cant ind 16
first item:  3.211   4.919
fitness order first item:  34.192

----> iteración:  3
cant ind 16
first item:  3.243   4.919
fitness order first item:  35.55799999999999

----> iteración:  4
cant ind 16
first item:  3.117   5.219
fitness order first item:  36.057

----> iteración:  5
cant ind 16
first item:  3.287   5.232
fitness order first item:  35.677

----> iteración:  6
cant ind 16
first item:  3.333   5.234
fitness order first item:  36.847

----> iteración:  7
cant ind 16
first item:  3.309   5.277
fitness order first item:  37.192

----> iteración:  8
cant ind 16
first item:  3.329   5.537
fitness order first item:  37.199

----> iteración:  9
cant ind 16
first item:  3.448   5.296
fitness order first item:  39.43

----> iteración:  10
cant ind 16
first item:  3.47   5.6

# Task 3
## Función:
$$f(x_1,x_2) = 5x_1 - x_1^2 + 8x_2 - 2x_2^2$$
## Restricciones:
$$3x_1 + 2x_2 \leq 6$$
$$x_1 \geq 0, x_2 \geq 0$$

In [558]:
fitness = lambda x1,x2: (5*x1) - (x1**2) + (8*x2) - (2*(x2**2))
restr = lambda x1: (6.0/2.0) - (3.0/2.0)*x1
o = Optimo(fitness, 15, 20, 0, 200, 0, restr, 100)
print("RESULTADOS: ")
print("x1: ",o.x1, "\tx2: ",o.x2)
print("fitness: ", o.fitness)


----> iteración:  0
cant ind 15
first item:  19.7   -10.7
fitness order first item:  -195.131969

----> iteración:  1
cant ind 16
first item:  2.5   -7.976
fitness order first item:  -89.43656100000001

----> iteración:  2
cant ind 16
first item:  4.1   -5.118
fitness order first item:  -87.084984

----> iteración:  3
cant ind 16
first item:  4.125   -5.118
fitness order first item:  -92.833539

----> iteración:  4
cant ind 16
first item:  2.447   -5.127
fitness order first item:  -85.42998700000001

----> iteración:  5
cant ind 16
first item:  2.535   -5.059
fitness order first item:  -84.17048599999998

----> iteración:  6
cant ind 16
first item:  2.368   -5.033
fitness order first item:  -83.220486

----> iteración:  7
cant ind 16
first item:  2.444   -4.903
fitness order first item:  -81.025737

----> iteración:  8
cant ind 16
first item:  2.257   -4.902
fitness order first item:  -80.501907

----> iteración:  9
cant ind 16
first item:  2.46   -4.659
fitness order first item:  -77